# Nonlinear feature transformation

Note this notebook is not meant as a didactic notebook for you to learn. It just contains the code for the feature transformation demo that was shown in the lecture. There is no need for you to understand the code. 

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

Generating data

In [ ]:
np.random.seed(42)
beta = 0.2

xs = np.array([-1.0, -0.9, 0.85, -0.8, -0.75, -0.1, 0.1, 0.2, 0.3, 0.5, 0.8, 0.9, 1.0])
def true_fun(xs): 
    return np.sin(2 * math.pi * xs) + 1

ys = true_fun(xs) + stats.norm(0, beta).rvs(len(xs))

In [ ]:
plt.plot(xs, ys, 'o')

x_values = np.linspace(-1, 1, 200)  
plt.plot(x_values,true_fun(x_values), '--r', label="ground truth")
  
plt.ylim([-2,4])

The following two functions are used to solve the regression and to plot the solution.  

In [ ]:

def solve_regression(transformation_functions, xs, ys):
    """ takes a list of transformation_functions as well as the training data (xs, ys) and solves the 
        corresponding regression problem using the normal equation """
    m = len(transformation_functions)
    
    # Matrix containing the basis functions evaluated at the input points
    Phi = np.zeros((len(xs), m))  
    for j in range(0, m):
        Phi[:,j] = transformation_functions[j](xs)
        
    # small additional regularization term to improve numerical stability
    reg = 1e-10 * np.eye(len(transformation_functions))
    w = np.linalg.inv(Phi.transpose() @ Phi + reg) @ Phi.transpose()

    return w @ ys

This function visualizes the solution

In [ ]:
def plot_solution(w, transformation_functions, xs, ys, show_basis = True, show_gt = True):
    """
    plots the basis function and the solutions for the (fitted) parameters w.
    xs and ys contain the data. 
    The parameter show_gt and show_basis determine whether the ground truth 
    and basis functions are plotted. 
    """
    # plot in the interval -1, 1
    x_values = np.linspace(-1, 1, 200)
    y_values = np.zeros(len(xs))
    
    # Compute the solution
    Phi = np.zeros((len(x_values), len(transformation_functions)))
    for j in range(0, len(transformation_functions)):
        Phi[:,j] = transformation_functions[j](x_values)
    y_values = Phi @ w
    
    # Plot the best prediction
    plt.plot(xs, ys, 'o')    
    plt.plot(x_values, y_values, 'k', label="predicted")

    
    # Plot the basis fucntions
    if show_basis:
        for j in range(0, len(transformation_functions)):
            plt.plot(x_values, transformation_functions[j](x_values) * w[j], ':', color="grey")
  
    if show_gt:
        plt.plot(x_values,true_fun(x_values), '--r', label="ground truth")
    
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.ylim([-2, 4])
    plt.legend()

Polynomials up to degree m 

In [ ]:
m = 0
basis_functions = [lambda x, i=i : np.power(x, i) for i in list(range(0,m+1))]
ws = solve_regression(basis_functions, xs, ys)
plot_solution(ws, basis_functions, xs, ys, show_gt = True)

Sigmoid functions

In [ ]:
m = 4
basis_functions = \
  [lambda x, i=i : 1 / (1 + np.exp(-(x-i)/0.02)) for i in np.linspace(-1,1,m)]
ws = solve_regression(basis_functions, xs, ys)
plot_solution(ws, basis_functions, xs, ys, show_gt = True)



Radial basis functions

In [ ]:
s = 0.1
basis_functions = [lambda x, mu=mu : np.exp(-np.power(x-mu, 2)/s) for mu in xs ]
ws = solve_regression(basis_functions, xs, ys)
plot_solution(ws, basis_functions, xs, ys, show_gt = True)